In [ ]:
'''
Author: Zala Reppmann, zala.reppmann@charite.de

Aim:
Investigating the effect of study site on the BOLD (Blood-Oxygen-Level-Dependent) response to the ScanSTRESS task
by calculating a ROI wise Bayes factor (Bayesian ANOVA).

Input:
- First level contrast image of the ScanSTRESS task (per participant)
- Brainnetome atlas for ROI definition

Output:
- Bayes Factor values (logBF10) for each ROI, indicating the impact of study site (neg = evidence for H0)
- NIFTI image mapping BFs back into the Brainnetome atlas

Steps:
- Setup logging
- Define paths and load necessary data, including the Brainnetome atlas and participant directories
- Exclude specified participant IDs from the analysis
- Prepare a masker using the NiftiLabelsMasker with the loaded atlas to extract time series data for each ROI
- For each participant, load the individual fMRI data, transform it using the masker, and store the ROI data
- Prepare the data and perform Bayes Factor calculations for each ROI using Bayesian ANOVA in R
- mapping Bayes Factors back to the atlas space and save as NIFTI image

Please note, that all paths and other identifying information have been removed from this script.
'''
#------------
# import libs
#------------
import os
import csv
import pandas as pd
import numpy as np
import nilearn
import nilearn.image as image
from nilearn import plotting, image
from nilearn.image import resample_to_img
from nilearn import datasets
from nilearn import masking
import rpy2.robjects as rob
from rpy2.rinterface_lib.embedded import RRuntimeError
from nilearn.input_data import NiftiLabelsMasker
import logging
from nilearn.image import new_img_like
from itertools import combinations

#--------------
# setup logging
#--------------
logging.getLogger('').handlers = [] # clear existing log handlers
logging.basicConfig(filename='/path/OBS_bayes_BF_ROI_logfile.log', level=logging.INFO, 
                    format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

#---------------------------
# define paths and load data
#---------------------------
# base path for data
basepath = '/path/results_fmri'

# Brainnetome atlas path
atlas_path = '/paths/BN246_atlas.nii.gz'

# excluded participant IDs
excluded_participant_ids = [] # expects list of strings containing participant IDs that are to be excluded from the analysis

# filter directories
all_participant_dirs = os.listdir(basepath)
participant_ids = [pid for pid in all_participant_dirs if pid not in excluded_participant_ids]

logging.info(f"Number of participants included: {len(participant_ids)}")
logging.info(f"Participant IDs: {', '.join(participant_ids)}")

#---------------
# Atlas business
#---------------
# load atlas
atlas_img = image.load_img(atlas_path)

# define path to reference image
reference_image_path = '/path/sub-XXXX_task-stress_feature-taskStress1_taskcontrast-stress_stat-effect_statmap.nii.gz'
reference_image = image.load_img(reference_image_path)

# resample atlas to match the reference image's space
resampled_atlas = image.resample_to_img(atlas_img, reference_image, interpolation='nearest')
atlas_img = resampled_atlas

#-----------------
# Extract ROI data
#-----------------
# load atlas and prep masker
masker = NiftiLabelsMasker(labels_img=atlas_img, standardize=True, memory='nilearn_cache', verbose=0)

# dictionary to hold ROI data per participant
participant_roi_data = {}

# extract data per participant
for pid in participant_ids:
    file_path = os.path.join(basepath, 'sub-{0}', 'func', 'task-stress', 'sub-{0}_task-stress_feature-taskStress1_taskcontrast-stress_stat-effect_statmap.nii.gz').format(pid.replace('sub-', ''))
    if os.path.exists(file_path):
        logging.info(f"Processing image for participant {pid}")
        participant_img = image.load_img(file_path)
        roi_time_series = masker.fit_transform(participant_img)
        participant_roi_data[pid] = roi_time_series
        logging.info(f"Data extraction complete for participant {pid}")
    else:
        logging.warning(f"Image file not found for participant {pid}")
        continue

logging.info("ROI data has been extracted for all participants.")

#-----------------------------------
# prep for Bayes Factor calculation
#-----------------------------------
# load BayesFactor package in R
rob.r('library(BayesFactor)')

# dictionary for study sites
site_dict = {'1': 'Berlin',
             '2': 'Mainz',
             '3': 'Nijmegen',
             '4': 'TelAviv',
             '5': 'Warsaw'}

site_labels = [site_dict[pid[5]] for pid in participant_ids]
rob.globalenv['site_labels'] = rob.StrVector(site_labels)

# prep data for R
rob.globalenv['participant_ids'] = rob.StrVector(participant_ids)
rob.globalenv['participant_sites'] = rob.StrVector([site_dict[pid[5]] for pid in participant_ids])

#----------------------------------
# ROI-wise Bayes Factor calculation
#----------------------------------
roi_labels = masker.labels_
roi_bayes_factors = {} 

for i, roi_name in enumerate(roi_labels):
    logging.info(f"Processing ROI {roi_name}")

    # gather data for current ROI across all participants
    roi_data = np.array([participant_roi_data[pid][:, i] for pid in participant_ids if pid in participant_roi_data])
    
    # check for NaN values and handle them appropriately
    if np.isnan(roi_data).any():
        roi_bayes_factors[roi_name] = np.nan
        logging.info(f"ROI {roi_name} contains NaN values and will be skipped for Bayes Factor calculation.")
        continue

    # prepare the data frame for R analysis
    rob.globalenv['roi_data'] = rob.FloatVector(roi_data.flatten())  # flatten to pass as a vector
    rob.r('df = data.frame(y = roi_data, id = rep(participant_ids, each=nrow(roi_data)/length(participant_ids)), group = factor(rep(participant_sites, each=nrow(roi_data)/length(participant_ids))))')
    
    # Bayesian ANOVA
    try:
        bf_result = rob.r('res <- tryCatch({anovaBF(y ~ group, data = df)@bayesFactor$bf}, error = function(e) NA)')
        if bf_result[0] is rob.NA_Real:
            roi_bayes_factors[roi_name] = np.nan
            logging.warning(f"Integration error in ROI {roi_name}. Assigned NaN.")
        else:
            roi_bayes_factors[roi_name] = bf_result[0]
            logging.info(f"Bayes Factor for ROI {roi_name}: {bf_result[0]}")
    except RRuntimeError as e:
        roi_bayes_factors[roi_name] = np.nan
        logging.error(f"R Runtime Error in ROI {roi_name}: {e}")

# log completion of Bayes Factor calculations
logging.info("Bayes Factor calculation completed for all ROIs.")

# -----------------------
# Bayesian post hoc tests
# -----------------------
groups = list(set(site_labels))
post_hoc_results = {roi: [] for roi in roi_labels if roi_bayes_factors.get(roi, 0) > 0}

def bayesian_t_test(group1_data, group2_data):
    # load BayesFactor package in R
    rob.r('library(BayesFactor)')

    # prep data for R
    rob.globalenv['group1_data'] = rob.FloatVector(group1_data)
    rob.globalenv['group2_data'] = rob.FloatVector(group2_data)
    
    # perform Bayesian one-sample t-test between the two groups
    try:
        # Compute Bayes factor for the difference between two groups
        bf_result = rob.r('ttestBF(x=group1_data, y=group2_data)')

        # Extract the Bayes Factor
        # extract the first element from the FloatVector
        bayes_factor = bf_result.slots['bayesFactor'][0][0]
        return bayes_factor
    except RRuntimeError as e:
        logging.error(f"R Runtime Error during Bayesian t-test: {e}")
        return np.nan  # return NaN if there's an error during the calculation
    except Exception as e:
        logging.error(f"General Error extracting Bayes Factor: {e}")
        return np.nan  # general catch for other issues


for roi_name in post_hoc_results.keys():
    # get index of current ROI
    roi_index = roi_labels.index(roi_name)
    
    # extract data for current ROI across all participants
    roi_data = np.array([participant_roi_data[pid][:, roi_index] for pid in participant_ids if pid in participant_roi_data])

    # create df from the ROI data
    df = pd.DataFrame(roi_data, columns=['data'])
    df['group'] = site_labels

    # perform pairwise Bayesian t-tests between groups
    for (group1, group2) in combinations(groups, 2):
        group1_data = df[df['group'] == group1]['data']
        group2_data = df[df['group'] == group2]['data']
        if not group1_data.empty and not group2_data.empty:
            bf = bayesian_t_test(group1_data, group2_data)
            post_hoc_results[roi_name].append((group1, group2, bf))
            logging.info(f"Bayesian t-test for ROI {roi_name} between {group1} and {group2}: BF={bf}")

#------------------
# save and clean up
#------------------
bf_values = [roi_bayes_factors.get(roi, np.nan) for roi in roi_labels]  # Use np.nan for any missing ROIs

# map BFs values back into atlas space
bf_image = masker.inverse_transform(bf_values)

# load the original atlas to create a mask of non-ROI regions (regions labeled as 0)
atlas_data = atlas_img.get_fdata()

# create a mask where atlas labels are 0 (non-ROI regions)
non_roi_mask = (atlas_data == 0)

# apply mask to set non-ROI regions in bf_image to NaN
bf_image_data = bf_image.get_fdata()
bf_image_data[non_roi_mask] = np.nan

# create a new Nifti image with updated BF data
bf_image_nan = new_img_like(bf_image, bf_image_data)

# save the updated BF image
bf_image_path = '/path/BFlog_ROI.nii'
bf_image_nan.to_filename(bf_image_path)

logging.info(f"Bayes Factor image with NaN in non-ROI areas saved as {bf_image_path}")

#------------------
# generate CSV file
#------------------
roi_bayes_df = pd.DataFrame(list(roi_bayes_factors.items()), columns=['ROI', 'Log(BF10)'])

#format post hoc results into a string
def format_post_hoc_results(results):
    # include only those results with positive BF -> evidence for H1
    return '; '.join([f"{g1} vs {g2}: BF={bf:.2f}" for g1, g2, bf in results if bf > 0])

# append formatted post hoc results to the DataFrame
roi_bayes_df['Post Hoc'] = roi_bayes_df['ROI'].apply(lambda x: format_post_hoc_results(post_hoc_results.get(x, [])))

# categorize evidence based on Log(BF10)
def evidence_category(log_bf):
    if log_bf > 4.61:
        return "Extreme evidence for H1"
    elif 3.40 <= log_bf <= 4.61:
        return "Very strong evidence for H1"
    elif 2.30 <= log_bf < 3.40:
        return "Strong evidence for H1"
    elif 1.10 <= log_bf < 2.30:
        return "Moderate evidence for H1"
    elif 0 <= log_bf < 1.10:
        return "Anecdotal evidence for H1"
    elif log_bf == 0:
        return "No evidence"
    elif -1.10 <= log_bf < 0:
        return "Anecdotal evidence for H0"
    elif -2.30 <= log_bf < -1.10:
        return "Moderate evidence for H0"
    elif -3.40 <= log_bf < -2.30:
        return "Strong evidence for H0"
    elif -4.61 <= log_bf < -3.40:
        return "Very strong evidence for H0"
    else:
        return "Extreme evidence for H0"

# add evidence category
roi_bayes_df['Evidence Category'] = roi_bayes_df['Log(BF10)'].apply(lambda x: evidence_category(x) if x is not None and not np.isnan(x) else "Data not available")
roi_bayes_df['Log(BF10)'] = roi_bayes_df['Log(BF10)'].apply(lambda x: x if x is not None and not np.isnan(x) else "NaN")

# load Brainnetome LUT data
LUT_path = "/path/BN_Atlas_246_LUT.csv"
lut_df = pd.read_csv(LUT_path)

# merge the data based on ROI matching with Label_ID
combined_df = pd.merge(roi_bayes_df, lut_df, left_on='ROI', right_on='Label_ID')

# save the updated csv
csv_path = '/path/BFlog_ROI_evidence.csv'
combined_df.to_csv(csv_path, index=False)

# Logging
logging.info(f"Evidence CSV with LUT data and post hoc results saved as {csv_path}")